# Studi Kasus Beer Production

# PROSES EKSEKUSI UNTUK EXPORT MODEL

## PROFILE KELOMPOK


NPM KETUA : 200411100153

NAMA KETUA : FIKRI AINUN NAJIB

NPM ANGGOTA : 200411100166

NAMA ANGGOTA : GHANIY ALBAR RASYIDI KUSUMA

GITHUB : https://github.com/LALA09-erha/kolaborasipro

DATASET : https://www.kaggle.com/datasets/shenba/time-series-datasets?select=monthly-beer-production-in-austr.csv


KELAS PSD C

Inisialisasi Libary dan beberapa Model

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import array
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor

import joblib



Pengambilan data

In [16]:
data = pd.read_csv("../Data/monthly-beer-production-in-austr.csv")

# add new data input to the dataframe
# data = data.append({'Month': '1995-09', 'Monthly beer production': 0}, ignore_index=True)
data = data ['Monthly beer production']


Split Data

In [17]:
data_training,data_test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=False, random_state=42)

In [18]:
data_training_new = pd.DataFrame(data_training)
data_test_new = pd.DataFrame(data_test)

Preprosessing Data

In [19]:
sc = MinMaxScaler()
train_scaled = sc.fit_transform(data_training_new)
test_scaled = sc.transform(data_test_new)

PROSES PENYIMPANAN NORMALISASI MIN MAX

In [20]:
filename = '../Model/norm.sav'
joblib.dump(sc, filename) 

['../Model/norm.sav']

fungsi Cross Validation

In [21]:
# Cross Validation
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)
# define univariate time series

Eksekusi Export Model Berdasarkan Model Terbaik Pada Training Model 

In [22]:
# inisialisasi variabel untuk menampung hasil akurasi dan n_steps
n_steps = range(6,7)
acc_fix = {}

for n in n_steps:
    # Proses Pemanggilan Cross Validation 
    X_train, y_train = split_sequence(train_scaled, n) 
    X_test, y_test = split_sequence(test_scaled, n) 

    # membuat kolom otomatis
    colom = []
    for c in range(n):
        if(c==0):
            colom.insert(0,"Xt")
        else:
            colom.insert(0,f"Xt-{c}")

    # # Masukkan ke dataframe hasil dari split  squens
    # UNTUK MEMISAHKAN X TRAIN, YTRAIN dan TEST
    newX_train = pd.DataFrame(np.reshape(X_train, (len(X_train) , n)),columns=colom)
    newY_train = pd.DataFrame(y_train, columns=['Output'])
    new_data_train = pd.concat([newX_train,newY_train],axis=1)

    newX_test = pd.DataFrame(np.reshape(X_test, (len(X_test) , n)),columns=colom)
    newY_test = pd.DataFrame(y_test, columns=['Output'])
    new_data_test = pd.concat([newX_test,newY_test],axis=1)


    # EKSEKUSI KE model

    # predict knn
    k_nei = range(1,30)
    acc = []
    for k in k_nei:
        if(k == 4  and n == 6):
            knn = KNeighborsRegressor(n_neighbors = k)
            knn.fit(newX_train,newY_train)
            #  Menyimpan model ke file eksternal
            filenameknn = f'../Model/knnk{k}n{n}.pk1' 
            joblib.dump(knn, filenameknn)

            # Proses Prediksi
            y_pred = knn.predict(newX_test)
            y_pred_shape = y_pred.reshape(-1,1)
            y_pred_inverse = sc.inverse_transform(y_pred_shape)
            newY_test_shape = y_test.reshape(-1,1)
            newY_test_inverse = sc.inverse_transform(newY_test_shape)
            mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
            print(f"KNN k = {k} n = {n} MAPE = {mape}")


    # Predict SVR dengan Bagging Regressor
    n_estimators = range(1,30)
    acc_svr = []
    for e in n_estimators:
        if(e == 2  and n == 6):
            regr = BaggingRegressor(SVR(), n_estimators=e, random_state=0).fit(newX_train, newY_train.values.ravel())
            # Menyimpan model ke file eksternal
            filenameregr = f'../Model/svre{e}n{n}.pk1'
            joblib.dump(regr, filenameregr)

            # Proses Prediksi
            y_pred = regr.predict(newX_test)
            y_pred_shape = y_pred.reshape(-1,1)
            y_pred_inverse = sc.inverse_transform(y_pred_shape)
            newY_test_shape = y_test.reshape(-1,1)
            newY_test_inverse = sc.inverse_transform(newY_test_shape)
            mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
            print(f"SVR n_estimators = {e} n = {n} MAPE = {mape}")


    # Predict dengan from RandomForestRegressor
    maxd = range(1,10)
    acc_rf = []
    for d in maxd:
        if(d == 8  and n == 6):
            regr = RandomForestRegressor(max_depth=d, random_state=0).fit(newX_train, newY_train.values.ravel())
            # Menyimpan model ke file eksternal
            filenamerf = f'../Model/rfd{d}n{n}.pk1'
            joblib.dump(regr, filenamerf)

            # Proses Prediksi
            y_pred = regr.predict(newX_test)
            y_pred_shape = y_pred.reshape(-1,1)
            y_pred_inverse = sc.inverse_transform(y_pred_shape)
            newY_test_shape = y_test.reshape(-1,1)
            newY_test_inverse = sc.inverse_transform(newY_test_shape)
            mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
            print(f"Random Forest max_depth = {d} n = {n} MAPE = {mape}")


KNN k = 4 n = 6 MAPE = 0.0707448021927608


SVR n_estimators = 2 n = 6 MAPE = 0.07518563792189382
Random Forest max_depth = 8 n = 6 MAPE = 0.07279313958249832
